In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import csv
import random
import itertools
import sys
import shutil
from tqdm import tqdm
import helpermethods
from edgeml_tf.trainer.protoNNTrainer import ProtoNNTrainer
from edgeml_tf.graph.protoNN import ProtoNN
import edgeml_tf.utils as utils
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from scipy.stats import uniform
from data_utils import *
import re
from mango.tuner import Tuner
import time
import pickle
import glob
from tensorflow import keras
import tensorflow.compat.v1.keras.backend as K

## Import Dataset

In [ ]:
f = '/home/nesl/earable_light/Activity_Dataset/' #Dataset directory
model_dir = 'trained_models/'
window_size = 550
stride = 50
channels = 2

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300,channels=2)
print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

## Feature Extraction

In [ ]:
feat_size = 10
featX_tr = np.zeros((X_tr.shape[0],feat_size))
featX_test = np.zeros((X_test.shape[0],feat_size))
for i in range(X_tr.shape[0]):
    cur_win = X_tr[i]
    featX_tr[i,0] = np.min(cur_win[:,0])
    featX_tr[i,1] = np.min(cur_win[:,1])
    featX_tr[i,2] = np.max(cur_win[:,0])
    featX_tr[i,3] = np.max(cur_win[:,1])
    featX_tr[i,4] = featX_tr[i,2]-featX_tr[i,0]
    featX_tr[i,5] = featX_tr[i,3]-featX_tr[i,1]
    featX_tr[i,6] = np.var(cur_win[:,0])
    featX_tr[i,7] = np.var(cur_win[:,1])
    featX_tr[i,8] = np.sqrt(featX_tr[i,6])
    featX_tr[i,9] = np.sqrt(featX_tr[i,7])  
    
for i in range(X_test.shape[0]):
    cur_win = X_test[i]
    featX_test[i,0] = np.min(cur_win[:,0])
    featX_test[i,1] = np.min(cur_win[:,1])
    featX_test[i,2] = np.max(cur_win[:,0])
    featX_test[i,3] = np.max(cur_win[:,1])
    featX_test[i,4] = featX_test[i,2]-featX_test[i,0]
    featX_test[i,5] = featX_test[i,3]-featX_test[i,1]
    featX_test[i,6] = np.var(cur_win[:,0])
    featX_test[i,7] = np.var(cur_win[:,1])
    featX_test[i,8] = np.sqrt(featX_test[i,6])
    featX_test[i,9] = np.sqrt(featX_test[i,7])
    
x_train = featX_tr
y_train = Y_tr
x_test = featX_test
y_test = Y_test
numClasses = Y_tr.shape[1]
dataDimension = x_train.shape[1]

mean = np.mean(x_train, 0)
std = np.std(x_train, 0)
std[std[:] < 0.000001] = 1
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std
Y_tr_int = np.argmax(Y_tr, axis=1)
Y_test_int = np.argmax(Y_test,axis=1)

shutil.rmtree('earable_dataset/', ignore_errors=True)
os.mkdir('earable_dataset/')
train = np.concatenate([Y_tr_int.reshape((Y_tr_int.shape[0],1)),x_train],axis=1)
with open('earable_dataset/train.npy','wb') as f:
    np.save(f,train)
test = np.concatenate([Y_test_int.reshape((Y_test_int.shape[0],1)),x_test],axis=1)
with open('earable_dataset/test.npy','wb') as f:
    np.save(f,test)

# Training and Hyperparameter Tuning

In [ ]:
NUM_EPOCHS = 3 #epochs to train each model for
bayesEpochs = 2 #epochs for hyperparameter tuning
log_file_name = 'Hyperparams_ProtoNN.csv'
if os.path.exists(log_file_name):
    os.remove(log_file_name)
if os.path.exists(log_file_name[0:-4]+'.p'):
    os.remove(log_file_name[0:-4]+'.p')
row_write = ['score', 'accuracy','Flash','PROJECTION_DIM','NUM_PROTOTYPES','GAMMA']
with open(log_file_name, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_write)

In [ ]:
def objective_NN(PROJECTION_DIM = 70, NUM_PROTOTYPES = 70, GAMMA = 0.004):
    
    #Fixed hyperparamters:
    
    REG_W = 0.000005
    REG_B = 0.0
    REG_Z = 0.00005
    SPAR_W = 0.8
    SPAR_B = 1.0
    SPAR_Z = 1.0
    LEARNING_RATE = 0.03
    BATCH_SIZE =32    
     
    W, B, gamma = helpermethods.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)
    
    
    X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
    Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
    protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                      NUM_PROTOTYPES, numClasses,
                      gamma, W=W, B=B)
    trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                             SPAR_W, SPAR_B, SPAR_Z,
                             LEARNING_RATE, X, Y, lossType='xentropy')
    sess = tf.Session()
    trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
                  printStep=600, valStep=1) 
    shutil.rmtree(model_dir, ignore_errors=True)
    os.mkdir(model_dir)
    acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
    W, B, Z, gamma = protoNN.getModelMatrices()
    matrixList = sess.run([W, B, Z])
    gamma = sess.run(gamma)
    sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
    nnz, size, sparse = helpermethods.getModelSize(matrixList, sparcityList, expected=False)
    np.save(model_dir + '/W.npy', matrixList[0])
    np.save(model_dir + '/B.npy', matrixList[1])
    np.save(model_dir + '/Z.npy', matrixList[2])
    np.save(model_dir + '/gamma.npy', gamma)
    
    model_size = size #flash usage
    accu = acc #accuracy
    
    score = 1.0*accu + 0.0*model_size #you can weigh the score to take into account model size too
    row_write = [score, accu,model_size,PROJECTION_DIM,NUM_PROTOTYPES,GAMMA]
    with open(log_file_name, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row_write)
        
    return score

In [ ]:
import pickle 

def save_res(data, file_name):
    pickle.dump(data, open(file_name, "wb" ))
    
param_dict = {
    'PROJECTION_DIM': np.arange(10,70),
    'NUM_PROTOTYPES': np.arange(10,70),
    'GAMMA': uniform(0.0015,0.05)
}

def objfunc(args_list):

    objective_evaluated = []
    
    start_time = time.time()
    
    for hyper_par in args_list:
        PROJECTION_DIM = hyper_par['PROJECTION_DIM']
        NUM_PROTOTYPES = hyper_par['NUM_PROTOTYPES']
        GAMMA = float(hyper_par['GAMMA'])
            
        objective = objective_NN(PROJECTION_DIM=PROJECTION_DIM,NUM_PROTOTYPES=NUM_PROTOTYPES,
                                 GAMMA=GAMMA)
        objective_evaluated.append(objective)
        
        end_time = time.time()
        print('objective:', objective, ' time:',end_time-start_time)
        
    return objective_evaluated

conf_Dict = dict()
conf_Dict['batch_size'] = 1 
conf_Dict['num_iteration'] = bayesEpochs
conf_Dict['initial_random']= 5
tuner = Tuner(param_dict, objfunc,conf_Dict)
all_runs = []
results = tuner.maximize()
all_runs.append(results)
save_res(all_runs,log_file_name[0:-4]+'.p')

# Train the best model

In [ ]:
PROJECTION_DIM = results['best_params']['PROJECTION_DIM']
NUM_PROTOTYPES = results['best_params']['NUM_PROTOTYPES']
GAMMA = float(results['best_params']['GAMMA'])

REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 0.8
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.03
BATCH_SIZE =32    

W, B, gamma = helpermethods.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                   NUM_PROTOTYPES, x_train)


X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
sess = tf.Session()
trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
              printStep=600, valStep=1) 
shutil.rmtree(model_dir, ignore_errors=True)
os.mkdir(model_dir)
acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
W, B, Z, gamma = protoNN.getModelMatrices()
matrixList = sess.run([W, B, Z])
gamma = sess.run(gamma)
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
nnz, size, sparse = helpermethods.getModelSize(matrixList, sparcityList, expected=False)
np.save(model_dir + '/W.npy', matrixList[0])
np.save(model_dir + '/B.npy', matrixList[1])
np.save(model_dir + '/Z.npy', matrixList[2])
np.save(model_dir + '/gamma.npy', gamma)

# Deployment

Restart Kernel before proceeding to prevent memory errors!

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import csv
import random
import itertools
import sys
import shutil
from tqdm import tqdm
import helpermethods
from edgeml_tf.trainer.protoNNTrainer import ProtoNNTrainer
from edgeml_tf.graph.protoNN import ProtoNN
import edgeml_tf.utils as utils
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from scipy.stats import uniform
from data_utils import *
import re
from mango.tuner import Tuner
import time
import pickle
import glob
from tensorflow import keras
import tensorflow.compat.v1.keras.backend as K

model_dir = 'trained_models/'

### Conversion to TFLite

In [ ]:
#saved in 'trainedModels/'
os.system("python3 protoNN_model_to_tflite.py --data-dir ./earable_dataset/ --output-dir "+model_dir)

### Conversion to C++

In [ ]:
os.system("xxd -i "+model_dir+"protoNN_model.tflite > model.cc")
with open('model.cc') as f:
    z = f.readlines()
f.close()   
z.insert(0,'#include "model.h"\n#ifdef __has_attribute\n#define HAVE_ATTRIBUTE(x) __has_attribute(x)\n#else\n#define HAVE_ATTRIBUTE(x) 0\n#endif\n#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))\n#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))\n#else\n#define DATA_ALIGN_ATTRIBUTE\n#endif\n')
z = [w.replace('trained_models_protoNN_model_tflite','protoNN_model') for w in z]
z = [w.replace('unsigned char protoNN_model[]','const unsigned char protoNN_model[] DATA_ALIGN_ATTRIBUTE') for w in z]
z[-1] = ""
my_f = open("model.cc","w")
for item in z:
    my_f.write(item)
my_f.close()


h_file_cont = ['#ifndef PROTONN_MODEL_H_\n',
           '#define PROTONN_MODEL_H_\n',
          'extern const unsigned char protoNN_model[];\n',
          '#endif\n']
my_f = open("model.h","w")
for item in h_file_cont:
    my_f.write(item)
my_f.close()